In [36]:
import pandas as pd
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from matplotlib import animation,rc

In [37]:
import plotly.plotly as py
import plotly.graph_objs as go

In [38]:
terror=pd.read_csv('globalterrorismdb_0617dist.csv',encoding='ISO-8859-1')

# renaming to give better names
terror.rename(columns={'iyear':'Year','imonth':'Month','iday':'Day','country_txt':'Country','region_txt':'Region','attacktype1_txt':'AttackType','target1':'Target','nkill':'Killed','nwound':'Wounded','summary':'Summary','gname':'Group','targtype1_txt':'Target_type','weaptype1_txt':'Weapon_type','motive':'Motive'},inplace=True)

# subset required columns
terror=terror[['Year','Month','Day','Country','Region','city','latitude','longitude','AttackType','Killed','Wounded','Group']]

# adding casualities 
terror['casualities']=terror['Killed']+terror['Wounded']

terror.head(3)

/Users/princegrover/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning:

Columns (4,6,31,33,53,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.



,Year,Month,Day,Country,Region,city,latitude,longitude,AttackType,Killed,Wounded,Group,casualities
0,1970,7,2,Dominican Republic,Central America & Caribbean,Santo Domingo,18.456792,-69.951164,Assassination,1.0,0.0,MANO-D,1.0
1,1970,0,0,Mexico,North America,Mexico city,19.432608,-99.133207,Hostage Taking (Kidnapping),0.0,0.0,23rd of September Communist League,0.0
2,1970,1,0,Philippines,Southeast Asia,Unknown,15.478598,120.599741,Assassination,1.0,0.0,Unknown,1.0


In [39]:
terror.shape

(170350, 13)

In [40]:
terror.dtypes

Year             int64
Month            int64
Day              int64
Country         object
Region          object
city            object
latitude       float64
longitude      float64
AttackType      object
Killed         float64
Wounded        float64
Group           object
casualities    float64
dtype: object

In [42]:
terror.isna().sum()

Year           0
Month          0
Day            0
Country        0
Region         0
city           0
latitude       0
longitude      0
AttackType     0
Killed         0
Wounded        0
Group          0
casualities    0
dtype: int64

In [43]:
terror.shape

(150159, 13)

Some basic anlysis

In [105]:
terror[terror['casualities'] == 8749]

,Year,Month,Day,Country,Region,city,latitude,longitude,AttackType,Killed,Wounded,Group,casualities
73054,2001,9,11,United States,North America,New York City,40.712784,-74.005941,Hijacking,1383.0,7366.0,Al-Qaida,8749.0


In [104]:
print('Country with Highest Terrorist Attacks:',terror['Country'].value_counts().index[0])
print('Regions with Highest Terrorist Attacks:',terror['Region'].value_counts().index[0])
print('Maximum casualities in an attack are:',terror['casualities'].max(),'that took place in',terror.loc[terror['casualities'].idxmax()].Country)

Country with Highest Terrorist Attacks: Iraq
Regions with Highest Terrorist Attacks: Middle East & North Africa
Maximum casualities in an attack are: 8749.0 that took place in United States


### Terrorism activities in India

In [15]:
import geoplotlib
from geoplotlib.utils import BoundingBox
from geoplotlib.colors import ColorMap
import json

In [16]:
terror.Year.min(), terror.Year.max()

(1970, 2016)

In [17]:
terror[:1]

,Year,Month,Day,Country,Region,city,latitude,longitude,AttackType,Killed,Wounded,Group,casualities
0,1970,7,2,Dominican Republic,Central America & Caribbean,Santo Domingo,18.456792,-69.951164,Assassination,1.0,0.0,MANO-D,1.0


In [ ]:
terror_sample = terror.copy()[['AttackType','latitude', 'longitude']]
terror_sample.columns = ['name','lat','lon']
terror_sample.dropna(inplace=True)

In [ ]:
geoplotlib.dot(terror_sample)
geoplotlib.show()

### Static map with number of attacks

In [15]:
terror_sample2 = terror.groupby(['city', 'latitude', 'longitude']).count()

In [16]:
terror_sample2.reset_index(inplace=True)

In [17]:
terror_sample2 = terror_sample2[['city', 'casualities', 'latitude', 'longitude']]
terror_sample2.columns = ['name', 'pop', 'lat', 'lon']

In [161]:
terror_sample2[:3]

,name,pop,lat,lon
5581,Baghdad,2091,33.300000,44.400000
33404,Lima,2006,-12.046378,-77.042791
27866,Karachi,1356,24.893379,67.028061


In [162]:
# only keep where pop>50 (i.e. atleast 10 attacks in 46 years of data)

terror_sample2 = terror_sample2[terror_sample2['pop'] > 10]

In [163]:
terror_sample2.shape

(1376, 4)

In [164]:
# terror_sample2['pop'] = terror_sample2['pop'].astype(int)

In [165]:
terror_sample2['pop'].min(), terror_sample2['pop'].max()

(11, 2091)

In [166]:
terror_sample2.sort_values(by='pop',  ascending=False, inplace=True)

In [167]:
terror_sample2[:2]

,name,pop,lat,lon
5581,Baghdad,2091,33.300000,44.400000
33404,Lima,2006,-12.046378,-77.042791


In [168]:
terror_sample2['text'] = terror_sample2['name'] + '<br>Total Attacks ' + (terror_sample2['pop']).astype(str)
limits = [(11,20),(21,50),(51,200),(201,1000),(1001,3000)]
colors = ["rgb(0,116,217)","green","rgb(133,20,75)","rgb(255,133,27)",'darkred']
cities = []
scale = 5

for i in range(len(limits)):
    lim = limits[i]
    terror_sample2_sub = terror_sample2[(terror_sample2['pop'] > lim[0]) & (terror_sample2['pop'] < lim[1]) ]
    city = dict(
        type = 'scattergeo',
        locationmode = 'World-map',
        lon = terror_sample2_sub['lon'],
        lat = terror_sample2_sub['lat'],
        text = terror_sample2_sub['text'],
        marker = dict(
            size = terror_sample2_sub['pop']/scale,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    cities.append(city)

layout = dict(
#         title = 'Total Terrorist Attacks from 1970-2016 <br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            projection=dict( type='Mercator' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
py.iplot( fig, validate=False, filename='d3-bubble-map-populations' )

### Animation world map

In [45]:
terror.shape

(150159, 13)

In [46]:
terror[:1]

,Year,Month,Day,Country,Region,city,latitude,longitude,AttackType,Killed,Wounded,Group,casualities
0,1970,7,2,Dominican Republic,Central America & Caribbean,Santo Domingo,18.456792,-69.951164,Assassination,1.0,0.0,MANO-D,1.0


In [47]:
terror_r = terror.groupby(by=['latitude','longitude', 'Year', 'city']).count()

In [48]:
terror_r = terror_r.reset_index()

In [49]:
terror_r = terror_r[['latitude', 'longitude', 'Year', 'casualities', 'city']]

In [50]:
terror_r[:2]

,latitude,longitude,Year,casualities,city
0,-53.154613,-70.916943,1984,3,Punta Arenas
1,-53.154613,-70.916943,1986,1,Punta Arenas


In [51]:
terror_r.shape

(82871, 5)

In [52]:
# reducing some rows

terror_r = terror_r[(terror_r.casualities > 5)]

In [53]:
terror_r.shape

(2868, 5)

In [54]:
# saving to load in r

terror_r.to_csv('terror_R.csv', index=False)

• attacks per year for all the countries   
• casualties  (killings+some thing else as per kernel) per year for all countries

In [55]:
terror_r[:1]

,latitude,longitude,Year,casualities,city
48,-38.734133,-72.584651,1988,14,Temuco


In [56]:
from datetime import datetime

In [57]:
chart_filename = "World Attacks " + str(datetime.now())

In [58]:
# list of years
years = terror_r["Year"].unique()
years = list(sorted(years.astype(str)))

# Group by year and count number of attacks
ylist = terror.groupby("Year").count()["casualities"]
ylist_cum = ylist.cumsum()

# Make running count of states
# statecount = df.groupby("on_year_s")["state"].nunique()
# statecount = list(statecount.astype(str))

# Get max range for subplot (minimum set to 0, no y-axis jump)
max_range = max(ylist) * 1.25
max_range_cum = max(ylist_cum) * 1.25

min_range_year = 1970
max_range_year = 2016
max_range_attacks =  terror_r['casualities'].max() # decides max of color scale
#max_range_states = 500
#max_range_statecount = 40

# Convert list items to string
ylist = list(ylist.astype(str))
ylist_cum = list(ylist_cum.astype(str))

In [59]:
terror_r.columns

Index(['latitude', 'longitude', 'Year', 'casualities', 'city'], dtype='object')

In [60]:
import numpy as np
import pandas as pd

import plotly.plotly as py
from plotly.graph_objs import *
from plotly.grid_objs import Grid, Column

In [61]:
# Since Grid has a size limit, it is good practice to upload multiple Grids for suplots in case of large datasets
grid_filename =  " Grid"
grid_filename2 = grid_filename + "2"

columns = []
columns2 = []

attack_avgs = []

for i, year in enumerate(years):

    # Filter df for current year only
    current_year = terror_r[terror_r["Year"] == int(year)]

    # Get average blade length for the year (ignores NaN values)
    attack_avg = current_year["casualities"].astype(float).mean()
    attack_avgs.append(attack_avg)

    lons = list(current_year["longitude"].astype(float))
    lats = list(current_year["latitude"].astype(float))
    texts = list(current_year["city"].astype(str)) # city names
    sizes = list(current_year["casualities"].astype(float)) # attacks numbers

    # Iteratively grow list to create running counts
    xvalues = years[:i + 1]
    yvalues = ylist[:i + 1]
    yvalues_cum = ylist_cum[:i + 1]

    # Idem for state count
#     statevalues = statecount[:i + 1]

    # Get data for Choropleth inlay
#     statelist = current_year.groupby("state").count()["unique_id"]
#     states = list(statelist.index.astype(str))
#     zvalues = list(statelist.astype(int))

    columns.append(Column(lons, "x{}".format(i + 1)))
    columns.append(Column(lats, "y{}".format(i + 1)))
    columns.append(Column(texts, "text{}".format(i + 1)))
    columns.append(Column(sizes, "size{}".format(i + 1)))

    columns2.append(Column(xvalues, "x{}".format(i + 1)))
    columns2.append(Column(yvalues, "y{}".format(i + 1)))
    columns2.append(Column(yvalues_cum, "y_cum{}".format(i + 1)))
#     columns2.append(Column(attack_avgs, "attack_avg{}".format(i + 1)))
#     columns2.append(Column(statevalues, "state{}".format(i + 1)))
#     columns2.append(Column(states, "location{}".format(i + 1)))
#     columns2.append(Column(zvalues, "z{}".format(i + 1)))

# Will throw error if file exists or path is not root
grid = Grid(columns)
py.grid_ops.upload(grid, grid_filename, auto_open=False)

grid2 = Grid(columns2)
# py.iplot(grid2, validate=False, filename='/Users/princegrover/Documents/msan/data_viz/assignments/project/DataVisualization-FinalProject/animation.html' )
py.grid_ops.upload(grid2, grid_filename2, auto_open=False)

'https://plot.ly/~groverpr/32/'

In [62]:
columns[3]

Column("[31.0, 9.0, 8.0, 7.0, 6.0, 6.0, 9.0, 8.0, 8.0, 12.0, 10.0, 6.0, 8.0, 6.0, 19.0, 7.0, 26.0, 8.0, 9.0, 7.0, 7.0, 11.0]", size1)

In [63]:
columns2[0]

Column("['1970']", x1)

In [79]:
manual = [[0.0, 'rgb(165,0,38)'], 
[0.1111111111111111, 'rgb(215,48,39)'],
[0.2222222222222222, 'rgb(244,109,67)'],
[0.3333333333333333, 'rgb(253,174,97)'], 
[0.4444444444444444, 'rgb(254,224,144)'],
[0.5555555555555556, 'rgb(224,243,248)'],
[0.6666666666666666, 'rgb(171,217,233)'], 
[0.7777777777777778, 'rgb(116,173,209)'],
[0.8888888888888888, 'rgb(69,117,180)'], 
[1.0, 'rgb(49,54,149)']]

blackpink = [[0, "rgb(59,37,73)"], [0.35, "rgb(76,43,96)"], [0.6, "rgb(93,49,119)"],
            [0.6, "rgb(109,54,143)"], [0.7, "rgb(143,66,189)"], [1, "rgb(152,80,200)"]]

In [90]:
# Main trace
trace1 = Scattermapbox(

    # GENERAL
    lonsrc = grid.get_column_reference("x1"),
    latsrc = grid.get_column_reference("y1"),
    textsrc = grid.get_column_reference("text1"),
    mode = "markers",
    hoverinfo = "lon+lat+text",

    # SPECS
    marker = dict(

        # BASIC
        sizesrc = grid.get_column_reference("size1"),
        colorsrc = grid.get_column_reference("size1"),
        sizeref = 4,
        opacity = "0.6",

        # COLORSCALE
        cmin = 0,
        cmax = max_range_attacks,
        autocolorscale = False,
        colorscale = manual,
        reversescale = True,
        colorbar = dict(
            title = "Number <br> of <br> Attacks", # colorbar on side for attack numbers max attacks
        ),
    ),

)

In [91]:
# Non-cumulative count
trace2 = Scatter(

    # GENERAL
    xsrc = grid2.get_column_reference("x1"),
    ysrc = grid2.get_column_reference("y1"),
    mode = "lines+markers",
    hoverinfo = "x+y",

    # SPECS
    line = dict(
        color = "#17BECF",
    ),
    marker = dict(
        symbol = "cross-thin-open",
    ),
    xaxis = "x",
    yaxis = "y4",

)

In [82]:
# Cumulative count
trace3 = Scatter(

    # GENERAL
    xsrc = grid2.get_column_reference("x1"),
    ysrc = grid2.get_column_reference("y_cum1"),
    mode = "lines",
    fill = "tozeroy",
    hoverinfo = "x+y",

    # SPECS
    line = dict(
        color = "rgb(17,123,215)",
    ),
    xaxis = "x2",
    yaxis = "y3",

)

Set up slider and buttons


In [92]:
animation_time = 1000
transition_time = 300
slider_transition_time = 300

slider = dict(

    # GENERAL
    plotlycommand = "animate",
    values = years,
    initialValue = years[0],
    visible = True,

    # ARGUMENTS
    args = [
        "slider.value",
        dict(
            duration = animation_time,
            ease = "cubic-in-out",
        ),
    ],

)

sliders = dict(

    # GENERAL
    active = 0,
    steps = [],

    currentvalue = dict(
        font = dict(size = 16),
        prefix = "Year : ",
        xanchor = "right",
        visible = True,
    ),
    transition = dict(
        duration = slider_transition_time,
        easing = "cubic-in-out",
    ),

    # PLACEMENT
    x = 0.1,
    y = 0,
    pad = dict(t = 40, b = 10),
    len = 0.9,
    xanchor = "left",
    yanchor = "top",

)

for year in years:

    slider_step = dict(

            # GENERAL
            method = "animate",
            value = year,
            label = year,

            # ARGUMENTS
            args = [
                [year],
                dict(
                    frame = dict(duration = animation_time, redraw = False),
                    transition = dict(duration = slider_transition_time),
                    mode = "immediate",
                    ),
                ],

            )

    sliders["steps"].append(slider_step)


updatemenus = dict(

    # GENERAL
    type = "buttons",
    showactive = False,
    x = 0.1, #x = 1.1
    y = 0, #y = 1
    pad = dict(t = 60, r = 10),
    xanchor = "right",
    yanchor = "top",
    direction = "left",

    # BUTTONS
    buttons=[
        dict(
            method = "animate",
            label = "Play",

            # PLAY
            args = [
                None,
                dict(
                    frame = dict(duration = animation_time, redraw = False), # False quicker but disables animations
                    fromcurrent = True,
                    transition = dict(duration = transition_time, easing = "quadratic-in-out"), # easing = "cubic-in-out"
                    mode = "immediate",
                    ),
                ],
            ),
        dict(
            method = "animate",
            label = "Pause",

            # PAUSE
            args = [
                [None], # Note the list
                dict(
                    frame = dict(duration = 0, redraw = False), # Idem
                    mode = "immediate",
                    transition = dict(duration = 0),
                    ),
                ],
            ),
        ],

)



Create layout


In [100]:
layout = dict(

    title = "Terrorist Attacks in the World (1970-2016)",

    # GENERAL LAYOUT
    width = 750,
    height = 700,
    autosize = True,
    font = dict(
        family = "Overpass",
        size = 12,
        color = "#000000",
    ),
    margin = dict(
        t = 50,
        l = 50,
        b = 50,
        r = 50,
        pad = 2,
    ),
    showlegend = False,
    hovermode = "closest",

    # ANIMATIONS
    slider = slider,
    sliders = [sliders],
    updatemenus = [updatemenus],

    annotations = [
        dict(
            text = "",
            x = 0.01,
            y = 0.2,
            align = "left",
            showarrow = False,
            xref = "paper",
            yref = "paper",
        )
    ],

    images = [
        dict(
            source="https://www.eecpress.com/wp-content/uploads/2017/12/cartoon-2-840x420.jpg",
            x = 0,
            y = 1.05,
            sizex = 0.10,
            sizey = 0.10,
            xref = "paper",
            yref = "paper",
            xanchor = "left",
            yanchor = "top"
      )
    ],

    # COLOR THEME
    plot_bgcolor = "#eaf2f2",
    paper_bgcolor = '#fcfcfc',

    # MAPBOX
    mapbox = dict(
#         accesstoken = mapbox_access_token,
        center = dict(
            lon = 50.38,
            lat = 34.16,
        ),
        zoom = 1.0,
#         style = "dark",
        domain = dict(
            x = [0, 1],
            y = [0.3, 0.95]
        ),
    ),

    # GEO PLOTS
    geo = dict(
        projection = dict(
            type = "Miller",
            scale = 1,
            #rotation = dict(lat = 0, lon = 0, roll = 0),
        ),
#         scope = "usa",
        showframe = False,
        resolution = "100",
#         bgcolor = "rgba(255, 255, 255, 0.0)",
        showland = True,
        showcoastlines = False,
#         landcolor = "rgb(229, 229, 229)",
        
        domain = dict(
            x = [0.82, 1],
            y = [0.20, 0.30]
        ),
    ),

    # AXIS (see current bug above)
    xaxis = dict(
        range = [min_range_year, max_range_year],
        domain = [0, 0.48],
        anchor = "y4",
        title = "Number of Attacks/ Year",
    ),
    yaxis4 = dict(
        range = [0, max_range],
        domain = [0, 0.22],
        anchor = "x",
    ),

    xaxis2 = dict(
        range = [min_range_year, max_range_year],
        domain = [0.52, 1],
        anchor = "y3",
        title = "Cumulative Attacks",
    ),
    yaxis3 = dict(
        range = [0, max_range_cum],
        domain = [0, 0.22],
        anchor = "x2",
    ),

)

Create frames


In [101]:
frames = []

for i, year in enumerate(years):

    # Create frame for each subplot
    frame_trace1 = dict(
        lonsrc = grid.get_column_reference("x{}".format(i + 1)),
        latsrc = grid.get_column_reference("y{}".format(i + 1)),
        textsrc = grid.get_column_reference("text{}".format(i + 1)),
        marker = dict(
            sizesrc = grid.get_column_reference("size{}".format(i + 1)),
            colorsrc = grid.get_column_reference("size{}".format(i + 1)),
        )
    )

    frame_trace2 = dict(
        xsrc = grid2.get_column_reference("x{}".format(i + 1)),
        ysrc = grid2.get_column_reference("y{}".format(i + 1)),
    )

    frame_trace3 = dict(
        xsrc = grid2.get_column_reference("x{}".format(i + 1)),
        ysrc = grid2.get_column_reference("y_cum{}".format(i + 1)),
    )

    frame = dict(
        data = [frame_trace1, frame_trace2, frame_trace3],
        name = year,
        traces = [0, 1, 2, 3, 4, 5],
    )

    frames.append(frame)

upload animations

In [102]:
data = [trace1, trace2, trace3]
figure = dict(data=data, layout=layout, frames=frames)
py.icreate_animations(figure, filename=chart_filename, auto_open=False)

### End